In [58]:
import pandas as pd
import zipfile
import json
from tqdm.auto import tqdm
import os
import sys
util_dir = os.path.abspath('../compass/utils')
sys.path.append(util_dir)
import utils
from lexicalrichness_v import LexicalRichness as lr

In [2]:
os.makedirs('jsonzip', exist_ok = True)
os.makedirs('output', exist_ok = True)

In [3]:
projects = input('Project(s): ').lower().strip()
project_list = utils.format_project_list(projects)

Project(s):  epsd2/literary, dsst


In [4]:
project_list = utils.oracc_download(project_list)

Saving http://oracc.org/dsst/json/dsst.zip as jsonzip/dsst.zip.


dsst: 0.00B [00:00, ?B/s]

Saving http://oracc.org/epsd2/literary/json/epsd2-literary.zip as jsonzip/epsd2-literary.zip.


epsd2/literary: 0.00B [00:00, ?B/s]

In [5]:
def parsejson(text, meta_d):
    lemmas = []
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            lemmas.extend(parsejson(JSONobject, meta_d))
        if "label" in JSONobject: 
            meta_d["label"] = JSONobject['label']   # `label` is the line number; it stays constant until
                                                    # the process move to a new line
        
        if JSONobject.get("type") == "field-start": # this is for sign lists, identifying fields such as
            meta_d["field"] = JSONobject["subtype"]  # sign, pronunciation, translation.
        elif JSONobject.get("type") == "field-end":
            meta_d.pop("field", None)                           # remove the key "field" to prevent it from being copied 
                                                              # to all subsequent lemmas (which may not have fields)
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            if "field" in meta_d:
                lemma["field"] = meta_d["field"]
            lemmas.append(lemma)
    return lemmas

In [6]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
for project in project_list:
    file = f'jsonzip/{project.replace("/", "-")}.zip'
    try:
        zip_file = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        errors = sys.exc_info() # get error information
        print(file), print(errors[0]), print(errors[1]) # and print it
        continue
    files = zip_file.namelist()     # list of all the files in the ZIP
    files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
    for filename in tqdm(files, desc = project):       #iterate over the file names
        id_text = project + filename[-13:-5] # id_text is, for instance, blms/P414332
        meta_d["id_text"] = id_text
        try:
            text_json_string = zip_file.read(filename).decode('utf-8')         #read and decode the json file of one particular text
            data_json = json.loads(text_json_string)                # make it into a json object (essentially a dictionary)
            lemm_l.extend(parsejson(data_json, meta_d))     # and send to the parsejson() function
        except:
            e = sys.exc_info() # get error information
            print(filename), print(e[0]), print(e[1]) # and print it
    zip_file.close()

dsst:   0%|          | 0/535 [00:00<?, ?it/s]

epsd2/literary:   0%|          | 0/915 [00:00<?, ?it/s]

In [7]:
words_df = pd.DataFrame(lemm_l)
words_df = words_df.fillna('')   # replace NaN (Not a Number) with empty string
words_df

,lang,form,delim,gdl,id_word,label,id_text,cf,gw,sense,norm0,pos,epos,base,morph,cont,norm,aform,field,stem
0,sux,edin-še₃,,"[{'v': 'edin', 'id': 'P356733.3.1.0', 'break':...",P356733.3.1,o 1’,dsst/P356733,,,,,,,,,,,,,
1,sux,mu-da,,"[{'v': 'mu', 'id': 'P356733.3.2.0', 'delim': '...",P356733.3.2,o 1’,dsst/P356733,,,,,,,,,,,,,
2,sux,igi,,"[{'v': 'igi', 'id': 'P356733.3.3.0', 'break': ...",P356733.3.3,o 1’,dsst/P356733,,,,,,,,,,,,,
3,sux,hu-mu-de₃-du₈,,"[{'v': 'hu', 'id': 'P356733.3.4.0', 'break': '...",P356733.3.4,o 1’,dsst/P356733,,,,,,,,,,,,,
4,sux,{ŋeš}al-e,,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",P356733.4.1,o 2’,dsst/P356733,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262727,sux,rib-x,,"[{'v': 'rib', 'id': 'P346641.8.3.0', 'break': ...",P346641.8.3,6',epsd2/literary/P346641,,,,,u,,,,,,,,
262728,sux,ši-in-x,,"[{'v': 'ši', 'id': 'P346641.8.4.0', 'delim': '...",P346641.8.4,6',epsd2/literary/P346641,,,,,u,,,,,,,,
262729,sux,x,,"[{'x': 'ellipsis', 'id': 'P346641.9.1.0', 'bre...",P346641.9.1,7ʾ,epsd2/literary/P346641,,,,,u,,,,,,,,
262730,sux,x,,"[{'v': 'x', 'id': 'P346641.9.2.0', 'break': 'd...",P346641.9.2,7ʾ,epsd2/literary/P346641,,,,,u,,,,,,,,


In [8]:
findreplace = {' ' : '-', ',' : ''}
words_df = words_df.replace({'gw' : findreplace, 'sense' : findreplace}, regex=True)

In [9]:
words_df = words_df[words_df.lang.str.startswith('sux')]

In [10]:
keep = ['form', 'cf', 'gw', 'pos', 'norm0', 'base', 'id_text']
words_df = words_df[keep]

In [11]:
dsst_id = [idtext[-7:] for idtext in words_df['id_text'] if idtext.startswith('dsst')]
dsst_id = list(set(dsst_id))
duplicates = [idtext for idtext in words_df['id_text'] if idtext[-7:] in dsst_id]
duplicates = list(set(duplicates))
duplicates = [idtext for idtext in duplicates if not idtext.startswith('dsst')]

In [12]:
words_df = words_df[~words_df.id_text.isin(duplicates)]

In [13]:
words_df

,form,cf,gw,pos,norm0,base,id_text
0,edin-še₃,,,,,,dsst/P356733
1,mu-da,,,,,,dsst/P356733
2,igi,,,,,,dsst/P356733
3,hu-mu-de₃-du₈,,,,,,dsst/P356733
4,{ŋeš}al-e,,,,,,dsst/P356733
...,...,...,...,...,...,...,...
262727,rib-x,,,u,,,epsd2/literary/P346641
262728,ši-in-x,,,u,,,epsd2/literary/P346641
262729,x,,,u,,,epsd2/literary/P346641
262730,x,,,u,,,epsd2/literary/P346641


In [14]:
words_l = []
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']
operators = ['&', '%', '@', '×']
for e in tqdm(words_df.form):
    word = []
    if '1(šar₂{gal})' in e: # this cheating but it seems to work (appears in SKL 38)
            e = e.replace('1(šar₂{gal})', '1(šar₂)-gal')
    for s in separators: # first split word into signs   
        e = e.replace(s, ' ').strip()
    s_l = e.split()
    for sign in s_l:
        if sign[0].isdigit(): # 1(geš₂), 2(DIŠ), etc.
            sign = sign.lower()
        elif sign[-1] == ')': # qualified sign - get only the qualifier
            stack = []  # |GIŠ×(GIŠ%GIŠ)|(LAK277) becomes LAK277
            ind = {}    # LAK277(|GIŠ×(GIŠ%GIŠ)|) becomes |GIŠ×(GIŠ%GIŠ)|
            for i, c in reversed(list(enumerate(sign))):
                if c == ')':
                    stack.append(i)
                if c == '(':
                    ind[stack.pop()] = i   # find the opening parens that belongs to the closing parens at position -1    
            start = ind[len(sign)-1]   # this line fails on 1(šar₂{gal}) in SKL.
            t = sign[start+1:-1]
            if t.isupper(): #leave 1(diš) etc. alone
                sign = t
            
        if '|' in sign:  # separate |DU.DU| and |DU+DU| into its components but not |DU&DU|
                        # and also not |DU.DU&DU|
            flag = False
            for o in operators:
                if o in sign:
                    flag = True
            if not flag:
                for s in separators2:
                    sign = sign.replace(s, ' ').strip() 
                sign_l = sign.split()
                word.extend(sign_l)
                continue
        elif "+" in sign:  # + as marker of gloss
            sign = sign.replace('+', ' ').strip()
            sign_l = sign.split()
            word.extend(sign_l)
            continue
        word.append(sign)
    words_l.append(word)      

  0%|          | 0/259788 [00:00<?, ?it/s]

In [15]:
words_df['signs'] = words_l

In [16]:
o = pd.read_pickle('ogsl.p', compression = None)

In [17]:
val = list(o["value"])
utf = list(o["utf8"])
names = list(o["name"])

In [18]:
d = dict(zip(names, utf))
d2 = dict(zip(val,names))

In [19]:
sign_l = []
for index, row in words_df.iterrows():
    for sign in row['signs']:
        n = [sign, row['id_text']]
        sign_l.append(n)

In [20]:
signs_df = pd.DataFrame(sign_l, columns = ['value', 'id_text'])
signs_df['sign_name'] = [d2.get(s.lower(), s) for s in signs_df.value]
signs_df['utf8'] = [d.get(n, n) for n in signs_df.sign_name]
signs_df = signs_df[['value', 'sign_name', 'utf8', 'id_text']]
signs_df = signs_df[signs_df.sign_name != 'X']
signs_df

,value,sign_name,utf8,id_text
0,edin,EDIN,𒂔,dsst/P356733
1,še₃,EŠ₂,𒂠,dsst/P356733
2,mu,MU,𒈬,dsst/P356733
3,da,DA,𒁕,dsst/P356733
4,igi,IGI,𒅆,dsst/P356733
...,...,...,...,...
575823,gi₄,GI₄,𒄄,epsd2/literary/P346641
575824,gin₇,DIM₂,𒁶,epsd2/literary/P346641
575825,rib,KAL,𒆗,epsd2/literary/P346641
575827,ši,IGI,𒅆,epsd2/literary/P346641


In [21]:
with open('aliases.txt') as a:
    alias = a.read().splitlines()

In [22]:
alias_d = {}

In [23]:
for line in alias:
    aliases = line.split()
    for value in aliases[1:]:
        alias_d[value] = aliases[0]

In [24]:
signs_df['value'] = [alias_d.get(value, value) for value in signs_df.value] 
signs_df

,value,sign_name,utf8,id_text
0,edin,EDIN,𒂔,dsst/P356733
1,še₃,EŠ₂,𒂠,dsst/P356733
2,ŋu₁₀,MU,𒈬,dsst/P356733
3,da,DA,𒁕,dsst/P356733
4,igi,IGI,𒅆,dsst/P356733
...,...,...,...,...
575823,gi₄,GI₄,𒄄,epsd2/literary/P346641
575824,gin₇,DIM₂,𒁶,epsd2/literary/P346641
575825,rib,KAL,𒆗,epsd2/literary/P346641
575827,ši,IGI,𒅆,epsd2/literary/P346641


In [25]:
SP_collections = {'Q000795' : 'SP1', 
            'Q000796' : 'SP2+6',
            'Q000797' : 'SP3',
            'Q000798' : 'SP4',
            'Q000799' : 'SP5',
            'Q000800' : 'SP7',
            'Q000801' : 'SP8',
            'Q000802' : 'SP9',
            'Q000803' : 'SP10', 
            'Q000804' : 'SP11',
            'Q000805' : 'SP12',
            'Q000806' : 'SP13',
            'Q000807' : 'SP14',
            'Q000808' : 'SP15', 
            'Q000809' : 'SP16',
            'Q000810' : 'SP17',
            'Q000811' : 'SP18',
            'Q000812' : 'SP19',
            'Q000813' : 'SP21',
            'Q000814' : 'SP22',
            'Q000815' : 'SP23',
            'Q000816' : 'SP24',
            'Q000817' : 'SP25',
            'Q000818' : 'SP26',
            'Q000819' : 'SP27',
            'Q000820' : 'SP28',
            'Q000821' : 'SP_Nippur',
            'Q000822' : 'SP_Susa',
            'Q000823' : 'SP_Ur',
            'Q000824' : 'SP_Uruk',
            'Q000825' : 'SP_Unknown'}           
            

In [26]:
signs_df['type'] = ['prov' if item[-7:] in SP_collections.keys() else 'lit' for item in signs_df.id_text  ]

In [27]:
signs_df

,value,sign_name,utf8,id_text,type
0,edin,EDIN,𒂔,dsst/P356733,lit
1,še₃,EŠ₂,𒂠,dsst/P356733,lit
2,ŋu₁₀,MU,𒈬,dsst/P356733,lit
3,da,DA,𒁕,dsst/P356733,lit
4,igi,IGI,𒅆,dsst/P356733,lit
...,...,...,...,...,...
575823,gi₄,GI₄,𒄄,epsd2/literary/P346641,lit
575824,gin₇,DIM₂,𒁶,epsd2/literary/P346641,lit
575825,rib,KAL,𒆗,epsd2/literary/P346641,lit
575827,ši,IGI,𒅆,epsd2/literary/P346641,lit


In [28]:
proverbs = signs_df[signs_df.type == 'prov']

In [29]:
literary = signs_df[signs_df.type != 'prov']

In [30]:
proverbs[['value', 'sign_name']].value_counts(ascending=True).reset_index(name='count')

,value,sign_name,count
0,hul₃,|GIŠ%GIŠ|,1
1,sag₁₀,|IGI.ERIN₂|,1
2,U₈,|LAGAB×(GUD&GUD)|,1
3,sagi,|SILA₃.ŠU.GABA|,1
4,saga₁₁,KIN,1
...,...,...,...
924,nu,NU,627
925,ba,BA,698
926,ŋu₁₀,MU,816
927,e,E,996


In [31]:
literary[['value', 'sign_name']].value_counts(ascending=True).reset_index(name='count')

,value,sign_name,count
0,ṭa₂,TA,1
1,LU₂.NE,LU₂.NE,1
2,su(2),su(2),1
3,MAN,|U.U|,1
4,si₂₃,GUL,1
...,...,...,...
2478,na,NA,10990
2479,ba,BA,11004
2480,e,E,12149
2481,ŋu₁₀,MU,16944


In [32]:
proverbs_s = proverbs.sample(n = 10000)
literary_s = literary.sample(n=10000)

In [33]:
proverbs_s[['value', 'sign_name']].value_counts(ascending=True).reset_index(name='count')

,value,sign_name,count
0,ir₉,GIR₃,1
1,gilim,|GI%GI|,1
2,asal₂,|A.TU.GABA.LIŠ|,1
3,azag,|KU₃.AN|,1
4,azalag₂,TUG₂,1
...,...,...,...
676,ga,GA,211
677,ba,BA,234
678,ŋu₁₀,MU,278
679,a,A,344


In [34]:
literary_s[['value', 'sign_name']].value_counts(ascending=True).reset_index(name='count')

,value,sign_name,count
0,$NI,$NI,1
1,ugu₆,KA,1
2,geme₂,|SAL.KUR|,1
3,ge(4),ge(4),1
4,ulušin,|ZIZ₂.A.AN|,1
...,...,...,...
787,ba,BA,221
788,na,NA,227
789,e,E,240
790,ŋu₁₀,MU,336


In [35]:
SP1 = signs_df[signs_df.id_text.str.contains('Q000795')]

In [36]:
SP1

,value,sign_name,utf8,id_text,type
195933,niŋ₂,GAR,𒃻,epsd2/literary/Q000795,prov
195934,gi,GI,𒄀,epsd2/literary/Q000795,prov
195935,na,NA,𒈾,epsd2/literary/Q000795,prov
195936,da,DA,𒁕,epsd2/literary/Q000795,prov
195937,a,A,𒀀,epsd2/literary/Q000795,prov
...,...,...,...,...,...
198710,ta,TA,𒋫,epsd2/literary/Q000795,prov
198711,šar₂,ŠAR₂,𒊹,epsd2/literary/Q000795,prov
198713,hi₂,GAN,𒃶,epsd2/literary/Q000795,prov
198714,a,A,𒀀,epsd2/literary/Q000795,prov


In [37]:
SP1[['value', 'sign_name']].value_counts(ascending=True).reset_index(name='count')

,value,sign_name,count
0,1,1,1
1,engar,APIN,1
2,er₁₀,DU,1
3,er₂,|A.IGI|,1
4,eŋir,EGIR,1
...,...,...,...
375,ba,BA,67
376,e,E,68
377,ŋu₁₀,MU,70
378,nu,NU,72


In [38]:
LiEB = signs_df[signs_df.id_text.str.contains("Q000482")]

In [39]:
LiEB[['value', 'sign_name']].value_counts(ascending=True).reset_index(name='count')

,value,sign_name,count
0,1(diš),DIŠ,1
1,išib,ME,1
2,iš,IŠ,1
3,isimu₂,GUL,1
4,ir,IR,1
...,...,...,...
188,ki,KI,18
189,ŋu₁₀,MU,18
190,li,LI,18
191,en,EN,26


In [40]:
decad = {'Q000395' : 'Shulgi A',
        'Q000481' : 'Lipit-Eshtar A',
        'Q000778' : 'Song of the Hoe',
        'Q000623' : 'Inana B',
        'Q000619' : 'Enlil A',
        'Q000751' : 'Kesh Temple Hymn',
        'Q000335' : "Enki's Journey to Nippur",
        'Q000339' : 'Inana and Ebih',
        'Q000736' : 'Nungal A',
        'Q000365' : 'Gilgamesh and Huwawa A'}


In [41]:
signs_decad = signs_df[signs_df.id_text.str[-7:].isin(decad.keys())]

In [42]:
signs_decad

,value,sign_name,utf8,id_text,type
160736,lugal,LUGAL,𒈗,epsd2/literary/Q000481,lit
160737,munus,SAL,𒊩,epsd2/literary/Q000481,lit
160738,dug₄,KA,𒅗,epsd2/literary/Q000481,lit
160739,ga,GA,𒂵,epsd2/literary/Q000481,lit
160740,šag₄,ŠA₃,𒊮,epsd2/literary/Q000481,lit
...,...,...,...,...,...
567560,d,AN,𒀭,epsd2/literary/Q000335,lit
567561,en,EN,𒂗,epsd2/literary/Q000335,lit
567562,ki,KI,𒆠,epsd2/literary/Q000335,lit
567563,zag,ZAG,𒍠,epsd2/literary/Q000335,lit


In [43]:
signs_decad[['value', 'sign_name']].value_counts(ascending=True).reset_index(name='count')

,value,sign_name,count
0,siškur₂,|AMAR×ŠE.AMAR×ŠE|,1
1,sim,NAM,1
2,lulim,|GIR₃×(LU.IGI)|,1
3,dab,DIB,1
4,idigna,|GU₂.GAR₃|,1
...,...,...,...
716,an,AN,380
717,en,EN,418
718,ba,BA,422
719,a,A,549


In [44]:
dec = []
for comp in decad.keys():
    c = signs_df[signs_df.id_text.str[-7:] == comp]
    l = len(c)
    s = len(set(c.sign_name))
    v = len(set(c.value))
    co = [decad[comp], l, round(s/l*100, 2), round(v/l*100, 2)]
    dec.append(co)

In [45]:
columns = ["composition", "length", "signs (%)","values (%)"]
pd.DataFrame(dec, columns=columns)

,composition,length,signs (%),values (%)
0,Shulgi A,1261,16.26,22.05
1,Lipit-Eshtar A,1086,17.96,22.74
2,Song of the Hoe,1207,15.91,19.80
3,Inana B,1721,12.09,15.63
4,Enlil A,1841,13.04,17.22
5,Kesh Temple Hymn,1740,12.01,14.89
6,Enki's Journey to Nippur,1368,14.62,18.64
7,Inana and Ebih,2111,11.27,14.50
8,Nungal A,1737,12.49,16.52
9,Gilgamesh and Huwawa A,3870,6.33,9.20


In [46]:
SP = []

In [47]:
for comp in SP_collections:
    c = signs_df[signs_df.id_text.str[-7:] == comp]
    l = len(c)
    s = len(set(c.sign_name))
    v = len(set(c.value))
    co = [SP_collections[comp], l, round(s/l*100, 2), round(v/l*100, 2)]
    SP.append(co)

In [48]:
columns = ["composition", "length", "TTR signs (%)","TTR values (%)"]
pd.DataFrame(SP, columns=columns)

,composition,length,TTR signs (%),TTR values (%)
0,SP1,2564,10.10,14.82
1,SP2+6,2922,9.24,13.48
2,SP3,2817,9.94,15.55
3,SP4,643,25.19,31.88
4,SP5,2296,10.02,14.11
5,SP7,932,20.92,27.90
6,SP8,1047,18.62,24.83
7,SP9,563,25.22,33.21
8,SP10,177,49.72,53.67
9,SP11,678,21.53,27.88


In [55]:
signs_by_comp = signs_df.groupby('id_text').aggregate({
    'value' : ' '.join, 
    'sign_name' : ' '.join, 
    'utf8' : ' '.join})

In [56]:
signs_by_comp

,value,sign_name,utf8
id_text,,,
dsst/P223416,a ba am₃ e ne nin kug zu dumu ŋu₁₀ u₃ ŋu₁₀ uŋ₃...,A BA |A.AN| E NE |SAL.TUG₂| KU₃ ZU TUR MU |IGI...,𒀀 𒁀 𒀀𒀭 𒂊 𒉈 𒎏 𒆬 𒍪 𒌉 𒈬 𒅇 𒈬 𒌦 𒉈 𒊏 𒂇 𒅇 𒌝 𒁺 𒇽 𒀀 𒊏 𒀊...
dsst/P223451,i de₃ ne da aŋ₂ e si ir rah₂ ri la₂ a ŋar rah₂...,I NE NE DA |NINDA₂×NE| E SI IR RA RI LAL A GAR...,𒄿 𒉈 𒉈 𒁕 𒉘 𒂊 𒋛 𒅕 𒊏 𒊑 𒇲 𒀀 𒃻 𒊏 𒉡 𒈛 𒄩 𒄙 𒊏 𒁀 𒀠 𒉡 𒁺 ...
dsst/P230180,da e₂ a mu₅ udun pa pah a mu₅ ig ge₂ sal la ŋe...,DA E₂ A NI |U.MU| PA LUL A NI IG KID SAL LA GI...,𒁕 𒂍 𒀀 𒉌 𒌋𒈬 𒉺 𒈜 𒀀 𒉌 𒅅 𒆤 𒊩 𒆷 𒄑 𒅅 𒂍 𒀝 𒀀 𒉌 𒉌𒌓 𒄯 𒁮 ...
dsst/P230446,la am₃ ŋen ne en ne en e AK i₃ gu₇ i₃ sar ma m...,LA |A.AN| DU NE EN NE EN E AK NI |KA×GAR| NI S...,𒆷 𒀀𒀭 𒁺 𒉈 𒂗 𒉈 𒂗 𒂊 𒀝 𒉌 𒅥 𒉌 𒊬 𒈠 𒈠 𒍑 𒈠 𒁶 𒍣 𒂵 𒂗 𒀀 𒇽...
dsst/P230736,u₃ um da a dug₄ sag₃ sag₃ ga NE EN mah ŋu₁₀ DU...,|IGI.DIB| UM DA A KA PA PA GA NE EN MAH MU DU ...,𒅇 𒌝 𒁕 𒀀 𒅗 𒉺 𒉺 𒂵 𒉈 𒂗 𒈤 𒈬 𒁺 𒀀 𒍪 𒃲 𒀀𒀀 𒈬 𒀭 𒂗 𒆤 𒇲 𒅗...
...,...,...,...
epsd2/literary/Q000825,ZI am₃ am₃ lu₂ a kam IG bi ki kug tuk šag₄ an ...,ZI |A.AN| |A.AN| LU₂ A |HI×BAD| IG BI KI KU₃ T...,𒍣 𒀀𒀭 𒀀𒀭 𒇽 𒀀 𒄰 𒅅 𒁉 𒆠 𒆬 𒌇 𒊮 𒀭 𒄾 𒊺 𒌇 𒄯 𒀭 𒊷 𒃻 𒌫 𒇹 ...
epsd2/literary/Q002338,kišib ŋu₁₀ sar ur DUN dam gar₃ rah₂ u₂ gu ba a...,MES MU SAR UR DUN DAM GAR₃ RA U₂ GU BA AN DE₂ ...,𒈩 𒈬 𒊬 𒌨 𒂄 𒁮 𒃼 𒊏 𒌑 𒄖 𒁀 𒀭 𒅗 𒁍 𒌔 𒊒 𒌝 𒈠 𒋫 𒂇 𒂊 𒋻 𒋻...
epsd2/literary/X010001,nin gal mah di an uraš a kug d nin i₃ si in ki...,|SAL.TUG₂| GAL MAH DI AN IB A KU₃ AN |SAL.TUG₂...,𒎏 𒃲 𒈤 𒁲 𒀭 𒅁 𒀀 𒆬 𒀭 𒎏 𒉌 𒋛 𒅔 𒆠 𒈾 𒌉 𒀭 𒈾 𒈨 𒌨 𒂵 𒀀𒀭 𒈪...


In [101]:
def stats(signs):
    signs = signs.split()
    lex = lr(signs) #, tokenizer = None, preprocessor = None)
    length = lex.words # number of lemmatized words
    lex_var = lex.terms # number of unique lemmas
    if length > 0:  # prevent division by 0
        ttr = lex.ttr
        mtld = lex.mtld()
    else:
        ttr = 0
        mtld = 0
    return [length, lex_var, ttr, mtld]

In [102]:
comp_v_all = []
for comp in signs_by_comp['value']:
    st = stats(comp)
    comp_v_all.append(st)
comp_s_all = []
for comp in signs_by_comp['sign_name']:
    st = stats(comp)
    comp_s_all.append(st)

In [111]:
comp_s_all_df = pd.DataFrame(comp_s_all , columns = 
                          ['length', 'unique_s', 's_ttr', 's_mtld']) 
                          # 'length2', 'unique_v', 'v_ttr', 'v_mtld'])
comp_v_all_df = pd.DataFrame(comp_v_all , columns = 
                          ['length', 'unique_v', 'v_ttr', 'v_mtld']) 
comp_v_all_df = comp_v_all_df[['unique_v', 'v_ttr', 'v_mtld']] 

In [112]:
comp_all_df = pd.concat([signs_by_comp, 
                         comp_s_all_df.set_index(signs_by_comp.index),
                        comp_v_all_df.set_index(signs_by_comp.index)], axis = 1).reset_index()
comp_all_df

,id_text,value,sign_name,utf8,length,unique_s,s_ttr,s_mtld,unique_v,v_ttr,v_mtld
0,dsst/P223416,a ba am₃ e ne nin kug zu dumu ŋu₁₀ u₃ ŋu₁₀ uŋ₃...,A BA |A.AN| E NE |SAL.TUG₂| KU₃ ZU TUR MU |IGI...,𒀀 𒁀 𒀀𒀭 𒂊 𒉈 𒎏 𒆬 𒍪 𒌉 𒈬 𒅇 𒈬 𒌦 𒉈 𒊏 𒂇 𒅇 𒌝 𒁺 𒇽 𒀀 𒊏 𒀊...,165,67,0.406061,20.999783,75,0.454545,23.419741
1,dsst/P223451,i de₃ ne da aŋ₂ e si ir rah₂ ri la₂ a ŋar rah₂...,I NE NE DA |NINDA₂×NE| E SI IR RA RI LAL A GAR...,𒄿 𒉈 𒉈 𒁕 𒉘 𒂊 𒋛 𒅕 𒊏 𒊑 𒇲 𒀀 𒃻 𒊏 𒉡 𒈛 𒄩 𒄙 𒊏 𒁀 𒀠 𒉡 𒁺 ...,34,28,0.823529,37.759883,29,0.852941,64.736000
2,dsst/P230180,da e₂ a mu₅ udun pa pah a mu₅ ig ge₂ sal la ŋe...,DA E₂ A NI |U.MU| PA LUL A NI IG KID SAL LA GI...,𒁕 𒂍 𒀀 𒉌 𒌋𒈬 𒉺 𒈜 𒀀 𒉌 𒅅 𒆤 𒊩 𒆷 𒄑 𒅅 𒂍 𒀝 𒀀 𒉌 𒉌𒌓 𒄯 𒁮 ...,358,115,0.321229,23.474123,136,0.379888,38.515792
3,dsst/P230446,la am₃ ŋen ne en ne en e AK i₃ gu₇ i₃ sar ma m...,LA |A.AN| DU NE EN NE EN E AK NI |KA×GAR| NI S...,𒆷 𒀀𒀭 𒁺 𒉈 𒂗 𒉈 𒂗 𒂊 𒀝 𒉌 𒅥 𒉌 𒊬 𒈠 𒈠 𒍑 𒈠 𒁶 𒍣 𒂵 𒂗 𒀀 𒇽...,802,151,0.188279,41.885029,188,0.234414,46.413591
4,dsst/P230736,u₃ um da a dug₄ sag₃ sag₃ ga NE EN mah ŋu₁₀ DU...,|IGI.DIB| UM DA A KA PA PA GA NE EN MAH MU DU ...,𒅇 𒌝 𒁕 𒀀 𒅗 𒉺 𒉺 𒂵 𒉈 𒂗 𒈤 𒈬 𒁺 𒀀 𒍪 𒃲 𒀀𒀀 𒈬 𒀭 𒂗 𒆤 𒇲 𒅗...,246,91,0.369919,35.142857,118,0.479675,46.115665
...,...,...,...,...,...,...,...,...,...,...,...
1428,epsd2/literary/Q000825,ZI am₃ am₃ lu₂ a kam IG bi ki kug tuk šag₄ an ...,ZI |A.AN| |A.AN| LU₂ A |HI×BAD| IG BI KI KU₃ T...,𒍣 𒀀𒀭 𒀀𒀭 𒇽 𒀀 𒄰 𒅅 𒁉 𒆠 𒆬 𒌇 𒊮 𒀭 𒄾 𒊺 𒌇 𒄯 𒀭 𒊷 𒃻 𒌫 𒇹 ...,1508,222,0.147215,34.512099,334,0.221485,44.691713
1429,epsd2/literary/Q002338,kišib ŋu₁₀ sar ur DUN dam gar₃ rah₂ u₂ gu ba a...,MES MU SAR UR DUN DAM GAR₃ RA U₂ GU BA AN DE₂ ...,𒈩 𒈬 𒊬 𒌨 𒂄 𒁮 𒃼 𒊏 𒌑 𒄖 𒁀 𒀭 𒅗 𒁍 𒌔 𒊒 𒌝 𒈠 𒋫 𒂇 𒂊 𒋻 𒋻...,108,58,0.537037,24.141325,64,0.592593,32.803318
1430,epsd2/literary/X010001,nin gal mah di an uraš a kug d nin i₃ si in ki...,|SAL.TUG₂| GAL MAH DI AN IB A KU₃ AN |SAL.TUG₂...,𒎏 𒃲 𒈤 𒁲 𒀭 𒅁 𒀀 𒆬 𒀭 𒎏 𒉌 𒋛 𒅔 𒆠 𒈾 𒌉 𒀭 𒈾 𒈨 𒌨 𒂵 𒀀𒀭 𒈪...,768,148,0.192708,31.014054,181,0.235677,38.176907
1431,epsd2/literary/X100010,nun d suen i din na nam ŋu₁₀ mah še₂₁ a zu u₄ ...,NUN AN |EN.ZU| I DIN NA NAM MU MAH |HU.NA₂| A ...,𒉣 𒀭 𒂗𒍪 𒄿 𒁷 𒈾 𒉆 𒈬 𒈤 𒄷𒈿 𒀀 𒍪 𒌓 𒉆 𒋾 𒆷 𒉌 𒁍 𒁍 𒌓 𒈬 𒃷 ...,97,54,0.556701,24.154018,55,0.567010,25.954722


In [113]:
proverbs = comp_all_df[comp_all_df.id_text.str[-7:].isin(SP_collections)]
proverbs = proverbs[['id_text', 'length', 'unique_s', 's_ttr', 's_mtld',
                    'unique_v', 'v_ttr', 'v_mtld']]
proverbs['title'] = [SP_collections.get(textid[-7:], textid) for textid in proverbs.id_text]

In [114]:
proverbs

,id_text,length,unique_s,s_ttr,s_mtld,unique_v,v_ttr,v_mtld,title
1398,epsd2/literary/Q000795,2564,259,0.101014,25.876639,380,0.148206,29.768397,SP1
1399,epsd2/literary/Q000796,2922,270,0.092402,23.800985,394,0.134839,25.643635,SP2+6
1400,epsd2/literary/Q000797,2817,280,0.099397,33.770444,438,0.155485,43.037849,SP3
1401,epsd2/literary/Q000798,643,162,0.251944,35.539647,205,0.318818,46.466939,SP4
1402,epsd2/literary/Q000799,2296,230,0.100174,30.520750,324,0.141115,35.104575,SP5
1403,epsd2/literary/Q000800,932,195,0.209227,31.122342,260,0.278970,37.592103,SP7
1404,epsd2/literary/Q000801,1047,195,0.186246,31.906878,260,0.248329,41.356500,SP8
1405,epsd2/literary/Q000802,563,142,0.252220,35.325490,187,0.332149,44.991050,SP9
1406,epsd2/literary/Q000803,177,88,0.497175,32.450000,95,0.536723,44.464806,SP10
1407,epsd2/literary/Q000804,678,146,0.215339,30.010493,189,0.278761,35.616830,SP11


In [115]:
decad_df = comp_all_df[comp_all_df.id_text.str[-7:].isin(decad)]
decad_df = decad_df[['id_text', 'length', 'unique_s', 's_ttr', 's_mtld',
                    'unique_v', 'v_ttr', 'v_mtld']]
decad_df['title'] = [decad.get(textid[-7:], textid) for textid in decad_df.id_text]

In [116]:
decad_df

,id_text,length,unique_s,s_ttr,s_mtld,unique_v,v_ttr,v_mtld,title
1060,epsd2/literary/Q000335,1368,200,0.146199,30.611427,255,0.186404,34.056801,Enki's Journey to Nippur
1064,epsd2/literary/Q000339,2111,238,0.112743,29.902815,306,0.144955,38.519133,Inana and Ebih
1087,epsd2/literary/Q000365,3870,245,0.063307,26.837479,356,0.091990,30.241760,Gilgamesh and Huwawa A
1114,epsd2/literary/Q000395,1261,205,0.162569,32.029351,278,0.220460,41.598550,Shulgi A
1180,epsd2/literary/Q000481,1086,195,0.179558,33.852805,247,0.227440,37.632655,Lipit-Eshtar A
1277,epsd2/literary/Q000619,1841,240,0.130364,30.431831,317,0.172189,37.397958,Enlil A
1280,epsd2/literary/Q000623,1721,208,0.120860,25.458694,269,0.156304,29.997495,Inana B
1367,epsd2/literary/Q000736,1737,217,0.124928,44.387416,287,0.165227,55.406699,Nungal A
1378,epsd2/literary/Q000751,1740,209,0.120115,27.001161,259,0.148851,29.008058,Kesh Temple Hymn
1385,epsd2/literary/Q000778,1207,192,0.159072,25.300690,239,0.198012,30.743164,Song of the Hoe
